# Cryptocurrency Price Prediction Project
This notebook collects historical data for top 10 cryptocurrencies using Twelve Data API, performs basic EDA, and prepares data for ML models.

In [ ]:
# Install necessary packages
!pip install requests pandas matplotlib scikit-learn

In [ ]:
# Import libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# API setup
API_KEY = '4f77610c1c0b4c92964487abd2616296'
BASE_URL = 'https://api.twelvedata.com/time_series'

In [ ]:
# Function to fetch historical data
def fetch_historical_data(symbol, interval, start_date, end_date):
    params = {
        'symbol': symbol,
        'interval': interval,
        'start_date': start_date,
        'end_date': end_date,
        'apikey': API_KEY,
        'format': 'JSON'
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    if 'values' in data:
        df = pd.DataFrame(data['values'])
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime')
        df = df.astype(float)
        return df
    else:
        print(f"Error for {symbol}: {data.get('message', 'Unknown error')}")
        return None

In [ ]:
# Top 10 cryptocurrencies
top_10_cryptos = ['BTC/USD', 'ETH/USD', 'BNB/USD', 'XRP/USD', 'ADA/USD', 
                   'SOL/USD', 'DOGE/USD', 'DOT/USD', 'MATIC/USD', 'LTC/USD']

In [ ]:
# Fetch and save data
interval = '1day'
start_date = '2023-01-01'
end_date = '2024-12-31'

crypto_data = {}

for crypto in top_10_cryptos:
    print(f"Fetching data for {crypto}...")
    data = fetch_historical_data(crypto, interval, start_date, end_date)
    if data is not None:
        crypto_data[crypto] = data
        filename = f"{crypto.replace('/', '_')}_historical_data.csv"
        data.to_csv(filename)
        print(f"Saved to {filename}")

In [ ]:
# Plot example for BTC
crypto = 'BTC/USD'
if crypto in crypto_data:
    plt.figure(figsize=(12, 6))
    plt.plot(crypto_data[crypto].index, crypto_data[crypto]['close'], label='Close Price')
    plt.title(f'Historical Price for {crypto}')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.show()
else:
    print("No data for BTC")

## LSTM Model for Price Prediction
In this section, we'll use an LSTM neural network to predict future cryptocurrency prices for 2025.

In [ ]:
# Install TensorFlow
!pip install tensorflow

In [ ]:
# Import additional libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
# Prepare data for LSTM
def prepare_lstm_data(data, sequence_length=60):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[['close']])
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])
    return np.array(X), np.array(y), scaler

In [ ]:
# Prepare LSTM inputs for BTC/USD
crypto = 'BTC/USD'
btc_data = crypto_data[crypto]
X, y, scaler = prepare_lstm_data(btc_data)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)

In [ ]:
# Predict future prices
last_sequence = X[-1]
future_predictions = []
current_sequence = last_sequence
for _ in range(365):  # Predict 365 days into the future
    next_pred = model.predict(np.expand_dims(current_sequence, axis=0), verbose=0)
    future_predictions.append(next_pred[0][0])
    current_sequence = np.append(current_sequence[1:], next_pred)[..., np.newaxis]

# Inverse transform predictions
predicted_prices = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

In [ ]:
# Plot predicted prices for 2025
plt.figure(figsize=(12,6))
plt.plot(predicted_prices, label='Predicted BTC Prices for 2025')
plt.title('BTC Price Prediction for 2025 using LSTM')
plt.xlabel('Days into 2025')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

## Sentiment Analysis on Crypto News Titles
We will perform sentiment analysis on recent crypto-related news titles using `TextBlob`.

In [ ]:
# Install TextBlob
!pip install textblob

In [ ]:
# Import TextBlob
from textblob import TextBlob

In [ ]:
# Fetch and analyze crypto news titles
def fetch_crypto_news():
    url = 'https://min-api.cryptocompare.com/data/v2/news/?categories=Cryptocurrency'
    response = requests.get(url)
    data = response.json()
    if 'Data' in data:
        news_items = data['Data']
        return [item['title'] for item in news_items]
    return []

news_titles = fetch_crypto_news()

# Analyze sentiment
sentiment_scores = [(title, TextBlob(title).sentiment.polarity) for title in news_titles]
df_sentiment = pd.DataFrame(sentiment_scores, columns=['Title', 'Sentiment'])
df_sentiment.head()

In [ ]:
# Visualize sentiment scores
plt.figure(figsize=(10, 5))
plt.hist(df_sentiment['Sentiment'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of News Sentiment')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Number of News Titles')
plt.grid(True)
plt.show()